In [ ]:
import os
import rasterio
import numpy as np

before = './images/before/bands'
after = './images/after/bands'

print('STD before')
for root, dirs, files in os.walk(before):
    for file in files:
        if file.endswith('.tif'):
            with rasterio.open(os.path.join(root, file)) as image_before:
                ndvi = np.array(image_before.read(1), dtype=image_before.read(1).dtype).astype(float)
                print(f'{file} Median NDVI: {np.std(ndvi)}')


print('STD after')
for root, dirs, files in os.walk(after):
    for file in files:
        if file.endswith('.tif'):
            with rasterio.open(os.path.join(root, file)) as image_before:
                ndvi = np.array(image_before.read(1), dtype=image_before.read(1).dtype).astype(float)
                print(f'{file} Median NDVI: {np.std(ndvi)}')



In [ ]:

import matplotlib.pyplot as plt

best_before = 'T30TWM_20220501T110619_NDVI.jp2.tif'
best_after = 'T30TWM_20220819T110619_NDVI.jp2.tif'

with rasterio.open(os.path.join(before, best_before)) as image_before:
    ndvi_before = np.array(image_before.read(1), dtype=image_before.read(1).dtype).astype(float)
    print(f'{best_before} Median NDVI: {np.std(ndvi)}')

    with rasterio.open(os.path.join(after, best_after)) as image_after:
        ndvi_after = np.array(image_after.read(1), dtype=image_before.read(1).dtype).astype(float)
        print(f'{best_after} Median NDVI: {np.std(ndvi)}')

    ndvi_diff = ndvi_after - ndvi_before

    ndvi_diff_profile = image_before.profile.copy()

    print(ndvi_diff.shape)
    ndvi_diff_profile.update({
        'dtype': 'float32',
        'nodata': 0,
        'height': ndvi_diff.shape[0],
        'width': ndvi_diff.shape[1],
    })

    with rasterio.open('ndvi_diff.tif', 'w', **ndvi_diff_profile) as dst:
        dst.write(ndvi_diff, 1)

    ndvi_diff[ndvi_diff>-0.1] = 1
    ndvi_diff[ndvi_diff<-0.1] = 0

    with rasterio.open('ndvi_diff_masked.tif', 'w', **ndvi_diff_profile) as dst:
        dst.write(ndvi_diff, 1)

    plt.imshow(ndvi_diff, cmap='RdYlGn')
    plt.colorbar()
    plt.title('Overview - Band 8 {}'.format(ndvi_diff.shape))


In [ ]:
from rasterio.features import shapes
from shapely.geometry import shape

from geopandas import GeoDataFrame

with rasterio.open('ndvi_diff_masked.tif') as src:
    data = src.read(1, masked=False)

    # Use a generator instead of a list
    shape_gen = ((shape(s), v) for s, v in shapes(data, transform=src.transform))

    # build a dict from unpacked shapes
    gdf = GeoDataFrame(dict(zip(["geometry", "class"], zip(*shape_gen))), crs=src.crs)
    gdf.to_file('./vectorized.gpkg', driver="GPKG")